# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.88,98,100,9.41,TF,1738716904
1,1,hermanus,-34.4187,19.2345,19.51,77,66,3.56,ZA,1738716905
2,2,grytviken,-54.2811,-36.5092,3.31,86,53,9.59,GS,1738716906
3,3,chonchi,-42.6128,-73.8092,13.95,81,0,0.39,CL,1738716907
4,4,klyuchi,52.2667,79.1667,-9.02,98,98,2.91,RU,1738716908


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    size='Humidity',
    colorbar=True,
    title='City Humidity Map'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df = city_data_df[
(city_data_df["Max Temp"] > 0) &
(city_data_df["Max Temp"] < 27) &
(city_data_df["Wind Speed"] < 4.5) &
(city_data_df["Cloudiness"] > 50) 
]

# Drop any rows with null values
narrow_df = narrow_df.dropna()

# Display sample data
narrow_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,hermanus,-34.4187,19.2345,19.51,77,66,3.56,ZA,1738716905
9,9,camana,-16.6228,-72.7111,23.12,84,68,2.31,PE,1738716914
11,11,port alfred,-33.5906,26.8910,22.88,96,100,1.34,ZA,1738716916
17,17,igbara-odo,7.5025,5.0626,23.42,85,79,1.28,NG,1738716922
19,19,benemerito de las americas,16.5167,-90.6439,21.68,94,96,1.23,MX,1738716924


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,hermanus,ZA,-34.4187,19.2345,77,
9,camana,PE,-16.6228,-72.7111,84,
11,port alfred,ZA,-33.5906,26.8910,96,
17,igbara-odo,NG,7.5025,5.0626,85,
19,benemerito de las americas,MX,16.5167,-90.6439,94,
...,...,...,...,...,...,...
520,siguiri,GN,11.6667,-9.5000,16,
526,nchelenge,ZM,-9.3451,28.7340,94,
527,wulingyuan,CN,29.3494,110.5441,68,
528,fenyi,CN,27.8112,114.6681,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [57]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng}, {lat}, {radius}"
    params["bias"] = f"proximity:{lng}, {lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hermanus - nearest hotel: No hotel found
camana - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
igbara-odo - nearest hotel: No hotel found
benemerito de las americas - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
don sak - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
calca - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
vila franca do campo - nearest hotel: No hotel found
kalemie - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
saint-denis - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel found
antalaha - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
imbituba - nearest hotel: No hotel found
mgandu - nearest hotel: No hotel found
noosa heads - nearest hotel: No hotel found
tadine - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
1,hermanus,ZA,-34.4187,19.2345,77,No hotel found
9,camana,PE,-16.6228,-72.7111,84,No hotel found
11,port alfred,ZA,-33.5906,26.8910,96,No hotel found
17,igbara-odo,NG,7.5025,5.0626,85,No hotel found
19,benemerito de las americas,MX,16.5167,-90.6439,94,No hotel found
...,...,...,...,...,...,...
520,siguiri,GN,11.6667,-9.5000,16,No hotel found
526,nchelenge,ZM,-9.3451,28.7340,94,No hotel found
527,wulingyuan,CN,29.3494,110.5441,68,No hotel found
528,fenyi,CN,27.8112,114.6681,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [59]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    size='Humidity',
    scale=0.5,
    color="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],
    colorbar=True,
    title='Hotels'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)